In [ ]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [ ]:
f = open(r"gme_titles.txt","r")


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in f:
    line = line.strip('\n')
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)





In [ ]:
print(results[:3])

In [ ]:
df = pd.DataFrame.from_records(results)
df.head()

In [30]:
import dbconn
import reddit_scraper
import pandas as pd

df = pd.DataFrame(reddit_scraper.search_pushshift_titles('GME',5,0),columns = ['Title','Context','Timestamp'])
df2 = pd.DataFrame(reddit_scraper.search_pushshift_titles('TSLA',5,0),columns = ['Title','Context','Timestamp'])
df3 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AMC',5,0),columns = ['Title','Context','Timestamp'])
df4 = pd.DataFrame(reddit_scraper.search_pushshift_titles('CLOV',5,0),columns = ['Title','Context','Timestamp'])
df5 = pd.DataFrame(reddit_scraper.search_pushshift_titles('PLTR',5,0),columns = ['Title','Context','Timestamp'])
df6 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AAPL',5,0),columns = ['Title','Context','Timestamp'])
df7 = pd.DataFrame(reddit_scraper.search_pushshift_titles('BB',5,0),columns = ['Title','Context','Timestamp'])
df8 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AMD',5,0),columns = ['Title','Context','Timestamp'])
df9 = pd.DataFrame(reddit_scraper.search_pushshift_titles('SNDL',5,0),columns = ['Title','Context','Timestamp'])
df10 = pd.DataFrame(reddit_scraper.search_pushshift_titles('PTON',5,0),columns = ['Title','Context','Timestamp'])


In [19]:
df_total = pd.concat([df,df2,df3,df4,df5,df6,df7,df8,df9,df10],ignore_index=True)

In [20]:
df_total.tail()

,Title,Context,Timestamp
995,Why I'm bearish on Peloton ($PTON),"First off, I love Peloton. I think their equip...",1613721411
996,PTON Calls,Peloton is on sale and I got a free Bike on th...,1613614368
997,Buy PTON 🚀 🌙,Peloton calls are hella cheap rn. For referenc...,1613584683
998,$TGYM vs $PTON 🚀🚀,[removed],1613480548
999,PTON was last year and what is this coming year?,[removed],1613249961


In [21]:
df_total2 = df_total[df_total.Context != "[removed]"]

In [22]:
df_total2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 885 entries, 0 to 997
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      885 non-null    object
 1   Context    885 non-null    object
 2   Timestamp  885 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 59.9+ KB


In [26]:
df_total3 = df_total2[df_total2.Context != ""]


In [28]:
df_total3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289 entries, 3 to 997
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      289 non-null    object
 1   Context    289 non-null    object
 2   Timestamp  289 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 9.0+ KB
